<a href="https://colab.research.google.com/github/leartssy/BlenderDiffuseNST/blob/main/DiffuseST_03_Tileability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Style Transfer by DiffuseST #

In [ ]:
!python --version

Python 3.11.13


In [ ]:
```python
!sudo apt-get update
!sudo apt-get install python3.11 python3.11-venv
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.11 1
!python3 --version
```

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip uninstall -y diffusers transformers accelerate peft huggingface-hub tokenizers

Found existing installation: diffusers 0.34.0
Uninstalling diffusers-0.34.0:
  Successfully uninstalled diffusers-0.34.0
Found existing installation: transformers 4.53.2
Uninstalling transformers-4.53.2:
  Successfully uninstalled transformers-4.53.2
Found existing installation: accelerate 1.8.1
Uninstalling accelerate-1.8.1:
  Successfully uninstalled accelerate-1.8.1
Found existing installation: peft 0.16.0
Uninstalling peft-0.16.0:
  Successfully uninstalled peft-0.16.0
Found existing installation: huggingface-hub 0.33.4
Uninstalling huggingface-hub-0.33.4:
  Successfully uninstalled huggingface-hub-0.33.4
Found existing installation: tokenizers 0.21.2
Uninstalling tokenizers-0.21.2:
  Successfully uninstalled tokenizers-0.21.2


In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/DiffustionST_v2/DiffuseST
!pip install -r requirements.txt
%cd ..

/content/drive/MyDrive/Colab Notebooks/DiffustionST_v2/DiffuseST
  Using cached accelerate-0.28.0-py3-none-any.whl.metadata (18 kB)
  Using cached peft-0.5.0-py3-none-any.whl.metadata (22 kB)
Using cached accelerate-0.28.0-py3-none-any.whl (290 kB)
Using cached peft-0.5.0-py3-none-any.whl (85 kB)
/content/drive/MyDrive/Colab Notebooks/DiffustionST_v2


In [ ]:
import torch

if torch.cuda.is_available():
    print("CUDA is available! Device name:", torch.cuda.get_device_name(0))
else:
    print("CUDA is not available. Using CPU.")

CUDA is available! Device name: Tesla T4


Load the blipdiffusion model

In [ ]:
#!git clone https://huggingface.co/salesforce/blipdiffusion

Cloning into 'blipdiffusion'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 32 (delta 0), reused 0 (delta 0), pack-reused 28 (from 1)
Unpacking objects: 100% (32/32), 520.37 KiB | 1.40 MiB/s, done.
Filtering content: 100% (6/6), 8.24 GiB | 58.03 MiB/s, done.
fatal: cannot exec '/content/drive/MyDrive/Colab Notebooks/DiffustionST_v2/blipdiffusion/.git/hooks/post-checkout': Permission denied


In [ ]:
#%cd blipdiffusion/

[Errno 2] No such file or directory: 'blipdiffusion/'
/content


Convert saved model.bin files to safetensors

In [ ]:
#from safetensors.torch import save_file
#import torch
#%cd "unet"
# Load the PyTorch .bin model
#state_dict = torch.load("diffusion_pytorch_model.bin", map_location="cpu")

# Save as .safetensors
#save_file(state_dict, "diffusion_pytorch_model.safetensors")

/content/drive/MyDrive/Colab Notebooks/DiffustionST_v2/blipdiffusion/unet


KeyboardInterrupt: 

In [ ]:
#%cd ..
#%cd "vae"
# Load the PyTorch .bin model
#state_dict = torch.load("diffusion_pytorch_model.bin", map_location="cpu")

# Save as .safetensors in the same folder
#save_file(state_dict, "diffusion_pytorch_model.safetensors")

In [ ]:
%cd ..
%cd ..

/content/drive/MyDrive/Colab Notebooks/DiffustionST_v2


In [ ]:
#!git clone https://github.com/huggingface/diffusers.git

In [ ]:
#%cd DiffuseST

[Errno 2] No such file or directory: 'DiffuseST'
/content/drive/MyDrive/Colab Notebooks


In [ ]:
#%cd /content/drive/MyDrive/Colab Notebooks/DiffustionST_v2/DiffuseST

/content/drive/MyDrive/Colab Notebooks/DiffustionST_v2/DiffuseST


In [ ]:
#run normal diffusion style transfer
#!python run.py

2025-11-25 08:32:40.420960: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764059560.441726    2536 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764059560.448037    2536 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764059560.464006    2536 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764059560.464031    2536 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764059560.464034    2536 computation_placer.cc:177] computation placer alr

# Adding Tileability by TexTile #

Install TexTile

In [ ]:
!pip install textile-metric

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 88.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 133.2 MB/s eta 0:00:00
  Created wheel for progressbar: filename=progressbar-2.5-py3-none-any.whl size=12065 sha256=df63452e66e26a92658736ea5c2335fc09b994bb4683f4f077154142c02fca1b
  Stored in directory: /root/.cache/pip/wheels/8d/bb/b2/5353b966ac6f3c5e1000629a9a5f6aed41794487f551e32efc
Successfully built progressbar


Using the TexTile metric as a loss function

In [ ]:
#initialize the textile loss
import textile
from textile.utils.image_utils import read_and_process_image

loss_textile = textile.Textile()

Next: the diffusion sampling loop must be modified to use this loss

run again with textile enabled

In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/DiffustionST_v2/DiffuseST

/content/drive/MyDrive/Colab Notebooks/DiffustionST_v2/DiffuseST


Textile_guidance_scale values:
0.1-1.0: low
2.0-5.0: medium
8.0+:high

In [ ]:
!python run.py --textile_guidance_scale 25.0 --alpha 0.4 --ddim_steps 50

2025-11-26 13:05:44.244268: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764162344.264026    2556 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764162344.270097    2556 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Loading pipeline components...:  86% 6/7 [01:20<00:11, 11.44s/it]/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(

tokenizer_config.json: 100% 48.0/48.0 [00:00<00:00, 442kB/s]

vocab.txt: 100% 232k/232k